In [ ]:
import requests
import json
import csv
import time
import pandas as pd
import numpy as np
import datetime
import os
import warnings
import boto3
from botocore.client import Config
from selectolax.parser import HTMLParser

Далее сформируем списки линий метро для сегментации запросов:

In [30]:
  # Сокольническая линия (красная ветка)
metro_list_Sokolnicheskaya = [
  'metro-bulvar-rokossovskogo', 'metro-cherkizovskaya', 'metro-preobrazhenskaya-ploshchad', 'metro-sokolniki', 'metro-krasnoselskaya'
, 'metro-komsomolskaya', 'metro-krasniye-vorota', 'metro-chistiye-prudy', 'metro-lubyanka', 'metro-okhotny-ryad'
, 'metro-biblioteka-imeni-lenina', 'metro-kropotkinskaya', 'metro-park-kultury', 'metro-frunzenskaya-1', 'metro-sportivnaya-1'
, 'metro-vorobyovy-gory', 'metro-universitet', 'metro-prospekt-vernadskogo', 'metro-yugo-zapadnaya', 'metro-troparyovo'
, 'metro-rumyantsevo', 'metro-salaryevo', 'metro-filatov-lug', 'metro-prokshino', 'metro-olhovaya'
, 'metro-kommunarka']

In [31]:
# Замоскворецкая линия (зеленая ветка)
metro_list_Zamoskvoretskaya = [
'metro-khovrino', 'metro-belomorskaya', 'metro-rechnoy-vokzal', 'metro-vodniy-stadion', 'metro-voikovskaya'
, 'metro-sokol', 'metro-aeroport', 'metro-dinamo', 'metro-belorusskaya', 'metro-mayakovskaya-1'
, 'metro-tverskaya', 'metro-teatralnaya', 'metro-novokuznetskaya', 'metro-paveletskaya', 'metro-avtozavodskaya'
, 'metro-tekhnopark', 'metro-kolomenskaya', 'metro-kashirskaya', 'metro-kantemirovskaya', 'metro-tsaritsino'
, 'metro-orekhovo', 'metro-domodedovskaya', 'metro-krasnogvardeyskaya', 'metro-alma-atinskaya']

In [32]:
#Арбатско-покровская линия (синяя ветка)
metro_list_Arbatsko_pokrovskaya = [
'metro-pyatnitskoye-shosse', 'metro-mitino', 'metro-volokolamskaya', 'metro-myakinino', 'metro-strogino'
, 'metro-krylatskoe', 'metro-molodezhnaya', 'metro-slavyanskiy-bulvar', 'metro-park-pobedy-1', 'metro-kievskaya'
, 'metro-smolenskaya', 'metro-arbatskaya', 'metro-ploshchad-revolyucii', 'metro-kurskaya', 'metro-baumanskaya' 
, 'metro-elektrozavodskaya', 'metro-semenovskaya', 'metro-partizanskaya', 'metro-izmaylovo', 'metro-pervomayskaya']

In [33]:
# Филевская линия (голубая ветка)
metro_list_Filevskaya = [
'metro-kuntsevskaya', 'metro-pionerskaya-1', 'metro-filyevskiy-park', 'metro-bagrationovskaya', 'metro-fili'
, 'metro-kutuzovskaya', 'metro-studencheskaya', 'metro-mejdunarodnaya', 'metro-delovoy-tsentr', 'metro-alexandrovski-sad'
, 'metro-arbatskaya-1']

In [34]:
# Кольцевая линия (коричневая ветка)
metro_list_Koltsevaya = [
'metro-dobryninskaya', 'metro-kazansky-railway-station', 'metro-krasnopresnenskaya', 'metro-novoslobodskaya']

In [35]:
# Калужско-рижская линия (оранжевая ветка)
metro_list_Kaluzhsko_righzkaya = [
'metro-medvedkovo', 'metro-babushkinskaya', 'metro-sviblovo', 'metro-botanicheskiy-sad', 'metro-vdnkh'
, 'metro-alekseevskaya', 'metro-rizhskaya', 'metro-prospekt-mira', 'metro-sukharevskaya', 'metro-turgenevskaya'
, 'metro-kitay-gorod', 'metro-tretyakovskaya', 'metro-oktyabrskaya', 'metro-shabolovskaya', 'metro-leninskiy-prospekt-1'
, 'metro-akademicheskaya-1', 'metro-profsoyuznaya', 'metro-noviye-cheremushki', 'metro-kaluzhskaya', 'metro-belyaevo'
, 'metro-konkovo', 'metro-tyopliy-stan', 'metro-yasenevo', 'metro-bitsevskiy-park', 'metro-schelkovskaya']

In [36]:
# Таганско-Краснопресненская линия (фиолетовая ветка)
metro_list_Tagansko_krasnopresnenskaya = [
'metro-planernaya', 'metro-skhodnenskaya', 'metro-tushinskaya', 'metro-spartak', 'metro-schukinskaya'
, 'metro-oktyabrskoe-pole', 'metro-polezhaevskaya', 'metro-begovaya-1', 'metro-ulitsa-1905-goda', 'metro-barrikadnaya'
, 'metro-pushkinskaya-1', 'metro-kuznetsky-most', 'metro-taganskaya', 'metro-proletarskaya-1', 'metro-volgogradskiy-prospekt'
, 'metro-textilschiki', 'metro-kuzminki', 'metro-ryazanskiy-prospekt', 'metro-zhulebino', 'metro-kotelniki']

In [37]:
# Калининская линия 
metro_list_Kalininskaya = [
'metro-marxistskaya', 'metro-ploshchad-ilyicha', 'metro-aviamotornaya', 'metro-shosse-entuziastov', 'metro-perovo'
, 'metro-novogireyevo', 'metro-novokosino', 'metro-minskaya', 'metro-lomonosovsky-prospekt', 'metro-ramenki'
, 'metro-michurinsky-prospekt', 'metro-ozernaya', 'metro-govorovo', 'metro-solntsevo', 'metro-borovskoe-shosse'
, 'metro-novoperedelkino', 'metro-rasskazovka']

In [38]:
# Серпуховско-Тимирязевская линия (серая ветка)
metro_list_Serpukhovsko_timiryazevskaya = [
'metro-altufyevo', 'metro-bibirevo', 'metro-otradnoe', 'metro-vladykino', 'metro-petrovsko-razumovskaya'
, 'metro-timiryazevskaya', 'metro-dmitrovskaya', 'metro-savyolovskaya', 'metro-mendeleevskaya', 'metro-tsvetnoy-bulvar'
, 'metro-chekhovskaya', 'metro-borovitskaya', 'metro-polyanka', 'metro-serpukhovskaya', 'metro-tulskaya'
, 'metro-nagatinskaya', 'metro-nagornaya', 'metro-nakhimovskiy-prospekt', 'metro-sevastopolskaya', 'metro-chertanovskaya'
, 'metro-yuzhnaya', 'metro-prazhskaya', 'metro-ulitsa-akademika-yangelya', 'metro-annino', 'metro-bulvar-dmitriya-donskogo']

In [39]:
# Люблинско-Дмитровская линия (салатовая ветка)
metro_list_Lublino_Dmitrovskaya = [
'mcd-lianozovo', 'metro-seligerskaya', 'metro-verkhniye-likhobory', 'metro-okruzhnaya', 'metro-fonvizinskaya'
, 'metro-butyrskaya', 'metro-marina-roscha', 'metro-dostoevskaya-1', 'metro-trubnaya', 'metro-sretenskiy-bulvar'
, 'metro-chkalovskaya-1', 'metro-rimskaya', 'metro-krestyanskaya-zastava', 'metro-dubrovka', 'metro-kozhukhovskaya'
, 'metro-pechatniki', 'metro-volzhskaya', 'metro-lyblino', 'metro-bratislavskaya', 'metro-maryino'
, 'metro-borisovo', 'metro-shipilovskaya', 'metro-zyablikovo']

In [40]:
# Большая кольцевая линия (бирюзовая ветка)
metro_list_Bolshaya_Koltsevaya = [
'metro-petrovsky-park', 'metro-cska', 'metro-khoroshevskaya', 'metro-narodnoye-opolcheniye', 'metro-mnevniki'
, 'metro-terekhovo', 'metro-davydkovo', 'metro-aminyevskaya', 'metro-novatorskaya', 'metro-vorontsovskaya'
, 'metro-zyuzino', 'metro-kakhovskaya', 'metro-varshavskaya', 'metro-klenovyy-bulvar', 'metro-nizhegorodskaya'
, 'metro-lefortovo', 'metro-lesoparkovaya', 'metro-ulitsa-starokachalovskaya', 'metro-ylitsa-skobelevskaya', 'metro-bulvar-admirala-ushakova'
, 'metro-ulitsa-corchakova', 'metro-buninskaya-alleya']

In [41]:
# Некрасовская линия (фиолетовая ветка)
metro_list_Nekrasovskaya = [
'metro-stakhanovskaya', 'metro-okskaya', 'metro-ulitsa-dmitrievskogo', 'metro-luhmanovskaya', 'metro-nekrasovka']

In [42]:
# Московское центральное кольцо
metro_list_MCK = [
'metro-rostokino', 'metro-belokamennaya', 'metro-lokomotiv', 'metro-sokolinaya-gora', 'metro-shosse-entuziastov-1'
, 'metro-andronovka', 'metro-novokhokhlovskaya', 'metro-ugreshskaya', 'metro-dubrovka-1', 'metro-zil'
, 'metro-verkhnie-kotly', 'metro-krymskaya', 'metro-ploshchad-gagarina', 'metro-luzhniki', 'metro-shelepikha'
, 'metro-khoroshyovo', 'metro-zorge', 'metro-panfilovskaya', 'metro-streshnevo', 'metro-baltiyskaya-1' 
, 'metro-koptevo', 'metro-likhobory']

In [43]:
# МЦД-1
metro_list_MCD1 = [
'mcd-odintsovo', 'mcd-bakovka', 'mcd-skolkovo', 'mcd-nemchinovka', 'mcd-setun'
, 'mcd-rabochiy-posyolok', 'mcd-testovskaya', 'mcd-degunino', 'mcd-mark', 'mcd-dolgoprudnaya'
, 'mcd-novodachnay', 'mcd-vodniki', 'mcd-khlebnikovo', 'mcd-sheremetyevskaya', 'mcd-lobnya']

In [44]:
# МЦД-2
metro_list_MCD2 = [
'mcd-podolsk', 'mcd-silikatnaya', 'mcd-ostafyevo', 'mcd-scherbinka', 'mcd-butovo'
, 'mcd-bitsa', 'mcd-krasniy-stroitel', 'mcd-pokrovskoye', 'mcd-moskvorechye', 'metro-kuryanovo-station'
, 'mcd-pererva', 'mcd-depo', 'mcd-kalitniki', 'mcd-moskva-tovarnaya', 'metro-ploschad-tryokh-vokzalov'
, 'mcd-grazhdanskaya', 'mcd-krasniy-baltiyets', 'mcd-trikotazhnaya', 'mcd-penyagino', 'mcd-pavshino'
, 'mcd-krasnogorskaya', 'mcd-opalikha', 'mcd-anikeevka', 'mcd-nakhabino']

In [45]:
# МЦД-3
metro_list_MCD3 = [
'metro-zelenograd-kryukovo', 'metro-firsanovskaya', 'metro-skhodnya', 'metro-podrezkovo', 'metro-novopodrezkovo'
, 'metro-molzhaninovo', 'metro-khimki', 'metro-grachevskaya', 'metro-mosselmash', 'metro-ostankino'
, 'metro-sortirovochnaya', 'metro-perovo-1', 'metro-plushevo', 'metro-veshnyaki', 'metro-ukhtomskaya'
, 'metro-lyubertsy', 'metro-panki', 'metro-tomilino', 'metro-kraskovo', 'metro-malakhovka'
, 'metro-udelnaya-1', 'metro-bykovo', 'metro-ilyinskaya', 'metro-otdykh']

In [46]:
# МЦД-4
metro_list_MCD4 = [
'metro-aprelevka', 'metro-pobeda-1', 'metro-kryokshino', 'metro-sanino', 'metro-kokoshkino'
, 'metro-tolstopaltsevo', 'metro-lesnoy-gorodok', 'metro-vnukovo', 'metro-michurinets', 'metro-peredelkino'
, 'metro-solnechnaya', 'metro-mescherskaya', 'metro-ochakovo', 'metro-matveyevskaya', 'metro-poklonnaya'
, 'metro-serp-i-molot', 'metro-chukhlinka', 'metro-kuskovo', 'metro-reutov', 'metro-nikolskoye'
, 'metro-saltykovskaya', 'metro-kuchino', 'metro-olgino', 'metro-zheleznodorozhnaya']

In [47]:
list_metrolines = [metro_list_Sokolnicheskaya, metro_list_Zamoskvoretskaya, metro_list_Arbatsko_pokrovskaya, metro_list_Filevskaya, metro_list_Koltsevaya, 
metro_list_Kaluzhsko_righzkaya, metro_list_Tagansko_krasnopresnenskaya, metro_list_Kalininskaya, metro_list_Serpukhovsko_timiryazevskaya, metro_list_Lublino_Dmitrovskaya, 
metro_list_Bolshaya_Koltsevaya, metro_list_Nekrasovskaya, metro_list_MCK, metro_list_MCD1, metro_list_MCD2, metro_list_MCD3, metro_list_MCD4]

In [49]:
# Структура извлекаемых данных
columns_yandex = [        'url_offer'
                        , 'url_offer_yand'
                        , 'seller'
                        , 'date_offer'
                        , 'rooms_offer'
                        , 'floors_house'
                        , 'floor_offer'
                        , 'type_offer'
                        , 'height_offer'
                        , 'square_total_offer'
                        , 'square_rooms_offer'
                        , 'square_kitchen_offer'
                        , 'room_space_1'
                        , 'room_space_2'
                        , 'room_space_3'
                        , 'room_space_4'
                        , 'price_offer'
                        , 'previous_price_offer'
                        , 'renovation_offer'
                        , 'address_offer'
                        , 'latitude'
                        , 'longitude'
                        , 'metro_name'
                        , 'metro_transp'
                        , 'time_to_metro'
                        , 'metro_latitude'
                        , 'metro_longitude'
                        , 'airport_id'
                        , 'airport_name'
                        , 'time_to_airport'
                        , 'airport_distance'
                        , 'airport_latitude'
                        , 'airport_longitude'
                        , 'school_id_1'
                        , 'school_name_1'
                        , 'school_address_1'
                        , 'time_to_school_1'
                        , 'school_distance_1'
                        , 'school_id_2'
                        , 'school_name_2'
                        , 'school_address_2'
                        , 'time_to_school_2'
                        , 'school_distance_2'
                        , 'park_id_1'
                        , 'park_name_1'
                        , 'time_to_park_1'
                        , 'park_distance_1'
                        , 'park_type_1'               
                        , 'park_id_2'
                        , 'park_name_2'
                        , 'time_to_park_2'
                        , 'park_distance_2'
                        , 'park_type_2'
                        , 'river_id'
                        , 'river_name'
                        , 'river_distance'
                        , 'time_to_river'
                        , 'dist_to_center'
                        , 'bathroom'
                        , 'balcony_type'
                        , 'window_view'
                        , 'built_year_offer'
                        , 'built_kvartal_offer'
                        , 'building_state'
                        , 'type_house_offer'
                        , 'autopark_offer'
                        , 'porchec_offer'
                        , 'lift_offer'
                        , 'rubbish_chute_offer'
                        , 'elite_offer'
                        , 'security_offer'
                        , 'description_offer'
                        , 'deal_status_offer'
                        , 'photo_list_offer'
                       ]

In [50]:
# Функция get_url_list получает первоначальную ссылку и формирует список ссылок (для переключения между страницами) 
def get_url_list(url):
    response = requests.get(url)
    response.encoding = 'utf-8'
    html = response.text
    tree = HTMLParser(html)
    script = tree.css_first('script[id="initial_state_script"]').text()
    script = script[23 : -1]
    data_pars = json.loads(script)
    number_of_pages = int((data_pars['seo']['offersCount'] // 20) + 1)
    if number_of_pages > 25:
        number_of_pages = 25
    list_url = []
    list_url.append(url)
    for page in range(2, number_of_pages + 1):
        list_url.append(url + '?page=' + str(page))
    return list_url

In [ ]:
# Функция pars_yandex возвращает данные по квартирам на 1 странице в виде датафрейма (в общем случае - 20 объявлений) 
def pars_yandex(url):
#    pass
    warnings.filterwarnings('ignore')
    response = requests.get(url)
    response.encoding = 'utf-8'
    html = response.text
    tree = HTMLParser(html)
    script = tree.css_first('script[id="initial_state_script"]')
    if script == None:
        return pd.DataFrame(columns = columns_yandex)
    else:
        script = script.text()

    script = script[23 : -1]
    data_pars = json.loads(script)
    offers = data_pars['map']['offers']['points']
    df_page = pd.DataFrame(columns = columns_yandex)
    for offer in offers:
# cсылка на первоисточник объявления
        url_offer = offer.get('url', 'Отсутствует ссылка')

# ссылка на объявление в яндекс-недвижимости: 
        url_offer_yand = offer.get('unsignedInternalUrl', 'Отсутствует ссылка') 

# Продавец (cобственник/агентство)
        if 'author' in offer:
            seller = offer['author'].get('category', None)
        else:
            seller = None    

# дата публикации объявления
        date_offer = offer.get('creationDate', 'Не указана дата публикации объявления')

# количество комнат в квартире
        rooms_offer = offer.get('roomsTotal', None)               

# этажность дома
        floors_house = offer.get('floorsTotal', None)                    

# этаж квартиры
        floor_offer = offer.get('floorsOffered', None)
        if type(floor_offer) == list:
            floor_offer = floor_offer[0]

# тип объявления (новостройка/вторичное жилье)
        type_offer = offer.get('flatType', 'Не указан тип объявления')                         

# высота потолков, м
        height_offer = offer.get('ceilingHeight', None)                   

 # площадь квартиры общая, м2
        if 'area' in offer:
            square_total_offer = offer['area'].get('value', None)
        else:
            square_total_offer =  None       

# площадь квартиры жилая, м2     
        if 'livingSpace' in offer:
            square_rooms_offer = offer['livingSpace'].get('value', None)
        else:    
            square_rooms_offer =  None

# площадь кухни, м2
        if 'kitchenSpace' in offer:
            square_kitchen_offer = offer['kitchenSpace'].get('value', None)
        else:
            square_kitchen_offer = None

# площади комнат, вносятся сведения по 4 комнатам
        if 'roomSpace' in offer:
            room_space_list = offer['roomSpace']
            if len(room_space_list) == 1:
                room_space_1 = room_space_list[0].get('value', None)
                room_space_2 = 0
                room_space_3 = 0
                room_space_4 = 0
            elif len(room_space_list) == 2:
                room_space_1 = room_space_list[0].get('value', None)
                room_space_2 = room_space_list[1].get('value', None)
                room_space_3 = 0
                room_space_4 = 0    
            elif len(room_space_list) == 3:
                room_space_1 = room_space_list[0].get('value', None)
                room_space_2 = room_space_list[1].get('value', None)
                room_space_3 = room_space_list[2].get('value', None)
                room_space_4 = 0
            elif len(room_space_list) >= 4:
                room_space_1 = room_space_list[0].get('value', None)
                room_space_2 = room_space_list[1].get('value', None)
                room_space_3 = room_space_list[2].get('value', None)
                room_space_4 = room_space_list[2].get('value', None)
            else:
                room_space_1 = None
                room_space_2 = None
                room_space_3 = None
                room_space_4 = None   
        else:
            room_space_1 = None
            room_space_2 = None
            room_space_3 = None
            room_space_4 = None           

# цена квартиры в объявлении, руб.
        if 'price' in offer:                 
            price_offer = offer['price'].get('value', None)
            previous_price_offer = offer['price'].get('previous', None)
        else:
            price_offer = None
            previous_price_offer = None

# ремонт квартиры
        if 'apartment' in offer:  
            renovation_offer = offer['apartment'].get('renovation', None)
        else:
            renovation_offer = None

# адрес квартиры в формате: город, улица, дом 
        if 'location' in offer:
            address_offer = offer['location'].get('geocoderAddress', None)
# координаты дома
            if 'point' in offer['location']:
                latitude = offer['location']['point'].get('latitude', None)
                longitude = offer['location']['point'].get('longitude', None)
            else:         
                latitude = None
                longitude = None
# характеристики метро: ближайшая станция, как добраться, время ходьбы до метро, координаты
            if 'metro' in offer['location']:
                metro_name = offer['location']['metro'].get('name', None)
                metro_transp = offer['location']['metro'].get('metroTransport', None)
                time_to_metro = offer['location']['metro'].get('timeToMetro', None)
                metro_latitude = offer['location']['metro'].get('latitude', None)
                metro_longitude = offer['location']['metro'].get('longitude', None)
            else:
                metro_name = None
                metro_transp = None
                time_to_metro = None
                metro_latitude = None
                metro_longitude = None
# Расположение до ближайшего аэропорта (парсится ближайший аэропорт из списка)               
            if 'airports' in offer['location']:
                if len(offer['location']['airports']) == 1:
                    airport_id = offer['location']['airports'][0].get('id', None)
                    airport_name = offer['location']['airports'][0].get('name', None)
                    time_to_airport = offer['location']['airports'][0].get('timeOnCar', None)
                    airport_distance = offer['location']['airports'][0].get('distanceOnCar', None)
                    airport_latitude = offer['location']['airports'][0].get('latitude', None)
                    airport_longitude = offer['location']['airports'][0].get('longitude', None)
                else:
                    airport_distance_list = []
                    for i in range(len(offer['location']['airports'])):
                        airport_distance_list.append(offer['location']['airports'][i].get('distanceOnCar', None))
                    min_ind = airport_distance_list.index(np.min(np.array(airport_distance_list)))    
                    airport_id = offer['location']['airports'][min_ind].get('id', None)
                    airport_name = offer['location']['airports'][min_ind].get('name', None)
                    time_to_airport = offer['location']['airports'][min_ind].get('timeOnCar', None)
                    airport_distance = offer['location']['airports'][min_ind].get('distanceOnCar', None)
                    airport_latitude = offer['location']['airports'][min_ind].get('latitude', None)
                    airport_longitude = offer['location']['airports'][min_ind].get('longitude', None)
            else:
                airport_id = None
                airport_name = None
                time_to_airport = None
                airport_distance = None
                airport_latitude = None
                airport_longitude = None

# Расположение до ближайших школ, парсятся две ближайшие школы из списка        
            if 'schools' in offer['location']:
                if len(offer['location']['schools']) == 1:
                    school_id_1 = offer['location']['schools'][0].get('schoolId', None)
                    school_name_1 = offer['location']['schools'][0].get('name', None)
                    school_address_1 = offer['location']['schools'][0].get('address', None)
                    time_to_school_1 = offer['location']['schools'][0].get('timeOnFoot', None)
                    school_distance_1 = offer['location']['schools'][0].get('distanceOnFoot', None)
                    school_id_2 = None
                    school_name_2 =  None
                    school_address_2 = None
                    time_to_school_2 = None
                    school_distance_2 = None

                elif len(offer['location']['schools']) == 2:
                    school_id_1 = offer['location']['schools'][0].get('schoolId', None)
                    school_name_1 = offer['location']['schools'][0].get('name', None)
                    school_address_1 = offer['location']['schools'][0].get('address', None)
                    time_to_school_1 = offer['location']['schools'][0].get('timeOnFoot', None)
                    school_distance_1 = offer['location']['schools'][0].get('distanceOnFoot', None)
                    school_id_2 = offer['location']['schools'][1].get('schoolId', None)
                    school_name_2 = offer['location']['schools'][1].get('name', None)
                    school_address_2 = offer['location']['schools'][1].get('address', None)
                    time_to_school_2 = offer['location']['schools'][1].get('timeOnFoot', None)
                    school_distance_2 = offer['location']['schools'][1].get('distanceOnFoot', None)

                else:
                    distance_school = []
                    for i in range(len(offer['location']['schools'])):
                        distance_school.append(offer['location']['schools'][i].get('distanceOnFoot', None))
                    min_ind_1, min_ind_2 = np.array(distance_school).argpartition(2)[:2]
                    school_id_1 = offer['location']['schools'][min_ind_1].get('schoolId', None)
                    school_name_1 = offer['location']['schools'][min_ind_1].get('name', None)
                    school_address_1 = offer['location']['schools'][min_ind_1].get('address', None)
                    time_to_school_1 = offer['location']['schools'][min_ind_1].get('timeOnFoot', None)
                    school_distance_1 = offer['location']['schools'][min_ind_1].get('distanceOnFoot', None)
                    school_id_2 = offer['location']['schools'][min_ind_2].get('schoolId', None)
                    school_name_2 = offer['location']['schools'][min_ind_2].get('name', None)
                    school_address_2 = offer['location']['schools'][min_ind_2].get('address', None)
                    time_to_school_2 = offer['location']['schools'][min_ind_2].get('timeOnFoot', None)
                    school_distance_2 = offer['location']['schools'][min_ind_2].get('distanceOnFoot', None)
            else:
                school_id_1 = None
                school_name_1 =  None
                school_address_1 = None
                time_to_school_1 = None
                school_distance_1 = None
                school_id_2 = None
                school_name_2 =  None
                school_address_2 = None
                time_to_school_2 = None
                school_distance_2 = None

# Расположение до ближайших парков, парсятся два ближайших парка из списка  
            if 'parks' in offer['location']:
                if len(offer['location']['parks']) == 1:
                    park_id_1 = offer['location']['parks'][0].get('parkId', None)
                    park_name_1 = offer['location']['parks'][0].get('name', None)
                    time_to_park_1 = offer['location']['parks'][0].get('timeOnFoot', None)
                    park_distance_1 = offer['location']['parks'][0].get('distanceOnFoot', None)
                    park_type_1 = offer['location']['parks'][0].get('parkType', None)
                    park_id_2 = None
                    park_name_2 = None
                    time_to_park_2 = None
                    park_distance_2 = None
                    park_type_2 = None

                elif len(offer['location']['parks']) == 2:
                    park_id_1 = offer['location']['parks'][0].get('parkId', None)
                    park_name_1 = offer['location']['parks'][0].get('name', None)
                    time_to_park_1 = offer['location']['parks'][0].get('timeOnFoot', None)
                    park_distance_1 = offer['location']['parks'][0].get('distanceOnFoot', None)
                    park_type_1 = offer['location']['parks'][0].get('parkType', None)
                    park_id_2 = offer['location']['parks'][1].get('parkId', None)
                    park_name_2 = offer['location']['parks'][1].get('name', None)
                    time_to_park_2 = offer['location']['parks'][1].get('timeOnFoot', None)
                    park_distance_2 = offer['location']['parks'][1].get('distanceOnFoot', None)
                    park_type_2 = offer['location']['parks'][1].get('parkType', None)
                else:
                    distance_park = []
                    for i in range(len(offer['location']['parks'])):
                        distance_park.append(offer['location']['parks'][i].get('distanceOnFoot', None))
                    min_ind_1, min_ind_2 = np.array(distance_park).argpartition(2)[:2]
                    park_id_1 = offer['location']['parks'][min_ind_1].get('parkId', None)
                    park_name_1 = offer['location']['parks'][min_ind_1].get('name', None)
                    time_to_park_1 = offer['location']['parks'][min_ind_1].get('timeOnFoot', None)
                    park_distance_1 = offer['location']['parks'][min_ind_1].get('distanceOnFoot', None)
                    park_type_1 = offer['location']['parks'][min_ind_1].get('parkType', None)
                    park_id_2 = offer['location']['parks'][min_ind_2].get('parkId', None)
                    park_name_2 = offer['location']['parks'][min_ind_2].get('name', None)
                    time_to_park_2 = offer['location']['parks'][min_ind_2].get('timeOnFoot', None)
                    park_distance_2 = offer['location']['parks'][min_ind_2].get('distanceOnFoot', None)
                    park_type_2 = offer['location']['parks'][min_ind_2].get('parkType', None)
            else:
                park_id_1 = None
                park_name_1 = None
                time_to_park_1 = None
                park_distance_1 = None
                park_type_1 = None                
                park_id_2 = None
                park_name_2 = None
                time_to_park_2 = None
                park_distance_2 = None
                park_type_2 = None

#водоёмы рядом с жильем (река/пруд)
            if 'ponds' in offer['location']:
                river_id = offer['location']['ponds'][0].get('pondId', None)
                river_name = offer['location']['ponds'][0].get('name', None)
                river_distance = offer['location']['ponds'][0].get('distanceOnFoot', None)
                time_to_river = offer['location']['ponds'][0].get('pondType', None)
            else:
                river_id = None
                river_name= None
                river_distance = None
                time_to_river = None

#Добавлено: расстояние до центра      
            if 'cityCenter' in offer['location']:
                dist_to_center = offer['location']['cityCenter'][0].get('distance', None)
            else:
                dist_to_center = None  

        else:    
            address_offer = None

            latitude = None
            longitude = None

            metro_name = None
            metro_transp = None
            time_to_metro = None
            metro_latitude = None
            metro_longitude = None

            airport_id = None
            airport_name = None
            time_to_airport = None
            airport_distance = None

            dist_to_center = None

            park_id_1 = None
            park_name_1 = None
            time_to_park_1 = None
            park_distance_1 = None
            park_type_1 = None
            park_id_2 = None
            park_name_2 = None
            time_to_park_2 = None
            park_distance_2 = None
            park_type_2 = None

            school_id_1 = None
            school_name_1 = None
            school_address_1 = None
            time_to_school_1 = None
            school_distance_1 = None
            school_id_2 = None
            school_name_2 = None
            school_address_2 = None
            time_to_school_2 = None
            school_distance_2 = None

            river_id = None
            river_name = None
            river_distance = None
            time_to_river = None

        if 'house' in offer:
#тип санузла (совмещенный/раздельный)
            bathroom = offer['house'].get('bathroomUnit', None)
#тип балкона:
            balcony_type = offer['house'].get('balconyType', None)      
#вид из окна (на улицу/во двор)
            window_view = offer['house'].get('windowView', None)
        else:
            bathroom = None
            balcony_type = None
            window_view = None
        
        if 'building' in offer:
# год постройки дома
            built_year_offer = offer['building'].get('builtYear', None)
# квартал сдачи дома (для новостроек)
            built_kvartal_offer = offer['building'].get('builtQuarter', None)
# статус строительства
            building_state = offer['building'].get('buildingState', None)

# тип дома
            type_house_offer = offer['building'].get('buildingType', None)

# тип автопарковки
            autopark_offer = offer['building'].get('parkingType', None)       

# количество подъездов в доме, шт.
            porchec_offer = offer['building'].get('porchesCount', None)

            if 'improvements' in offer['building']:
# наличие лифта
                lift_offer = offer['building']['improvements'].get('LIFT', None)

# наличие мусоропровода
                rubbish_chute_offer = offer['building']['improvements'].get('RUBBISH_CHUTE', None)

# маркер элитного жилья
                elite_offer = offer['building']['improvements'].get('ELITE', None)

# консьерж
                security_offer = offer['building']['improvements'].get('SECURITY', None)
            else:
                lift_offer = None
                rubbish_chute_offer = None
                elite_offer = None
                security_offer = None    

        else:
            built_year_offer = None
            built_kvartal_offer = None
            building_state = None
            type_house_offer = None
            autopark_offer = None
            porchec_offer = None
            lift_offer = None
            rubbish_chute_offer = None
            elite_offer = None
            security_offer = None

# описание квартиры от продавца
        description_offer = offer.get('description', None)

# статус объявления
        deal_status_offer = offer.get('dealStatus', None)        

# список ссылок на фотографии
        photo_list_offer = offer.get('fullImages', None)

        df_page.loc[len(df_page)] = [url_offer, url_offer_yand, seller, date_offer, rooms_offer
                        , floors_house, floor_offer, type_offer, height_offer, square_total_offer
                        , square_rooms_offer, square_kitchen_offer, room_space_1, room_space_2, room_space_3
                        , room_space_4, price_offer, previous_price_offer, renovation_offer, address_offer
                        , latitude, longitude, metro_name, metro_transp, time_to_metro
                        , metro_latitude, metro_longitude, airport_id, airport_name, time_to_airport
                        , airport_distance, airport_latitude, airport_longitude, school_id_1, school_name_1
                        , school_address_1, time_to_school_1, school_distance_1, school_id_2, school_name_2
                        , school_address_2, time_to_school_2, school_distance_2, park_id_1, park_name_1
                        , time_to_park_1, park_distance_1, park_type_1, park_id_2, park_name_2
                        , time_to_park_2, park_distance_2, park_type_2, river_id, river_name
                        , river_distance, time_to_river, dist_to_center, bathroom, balcony_type
                        , window_view, built_year_offer, built_kvartal_offer, building_state, type_house_offer
                        , autopark_offer, porchec_offer, lift_offer, rubbish_chute_offer, elite_offer
                        , security_offer, description_offer, deal_status_offer, photo_list_offer]
    
    return df_page

In [52]:
#Данная функция получает на вход список названий метро и возвращает список ссылок для парсинга по 4 подзапроса на каждую станцию метро: отдельно по новостройкам и вторичному рынке, а также по возрастанию и убыванию цены
def segment_url(list_metro):
    url_list_metro_extended = []
    url_base = 'https://realty.yandex.ru/moskva_i_moskovskaya_oblast/kupit/kvartira/'
    type_offer_list = ['/novostroyki', '/vtorichniy-rynok']
    for metro_station in list_metro:
        for type_offer in type_offer_list:
            url = url_base + metro_station + type_offer
            response = requests.get(url)
            response.encoding = 'utf-8'
            html = response.text
            tree = HTMLParser(html)
            script = tree.css_first('script[id="initial_state_script"]').text()
            script = script[23 : -1]
            data_pars = json.loads(script)
            number_of_offers = int(data_pars['seo']['offersCount'])
            if number_of_offers <= 500:
                url_list_metro_extended.append(url)
            else:
                url_list_metro_extended.append(url + '/?sort=PRICE')
                url_list_metro_extended.append(url + '/?sort=PRICE_DESC')
    return url_list_metro_extended

In [53]:
#Основная функция, получает на вход список метро для парсинга
def main(list_metro):
    url_list_metro_extended = segment_url(list_metro) 
    for url in url_list_metro_extended:    
        start_time = time.time()
        url_list = get_url_list(url)
        df_result_yand = pd.DataFrame(columns = columns_yandex)
        for url_lower_level in url_list:
            print(url_lower_level)
            df_page = pars_yandex(url_lower_level)
            df_result_yand = pd.concat([df_result_yand, df_page], axis = 0, join = 'outer')
        name_metro = url_lower_level[68:]
        name_metro = name_metro[:name_metro.find('/')]
        current_time = str(datetime.datetime.now().time())
        current_time = current_time[:-7]
        name_csv = 'Yandex_' + str(name_metro) + '_' + str(url_list_metro_extended.index(url)) + '_' + str(datetime.date.today().isoformat()) +'.csv'
        df_result_yand.to_csv(name_csv, encoding = 'utf-8')
        end_time = time.time()
        print(end_time - start_time)

In [27]:
list_metrolines = [metro_list_Sokolnicheskaya, metro_list_Zamoskvoretskaya, metro_list_Arbatsko_pokrovskaya, metro_list_Filevskaya, metro_list_Koltsevaya, 
metro_list_Kaluzhsko_righzkaya, metro_list_Tagansko_krasnopresnenskaya, metro_list_Kalininskaya, metro_list_Serpukhovsko_timiryazevskaya, metro_list_Lublino_Dmitrovskaya, 
metro_list_Bolshaya_Koltsevaya, metro_list_Nekrasovskaya, metro_list_MCK, metro_list_MCD1, metro_list_MCD2, metro_list_MCD3, metro_list_MCD4]

In [ ]:
directory = directory
os.chdir(directory)
files = os.listdir(directory)
df_result_yand = pd.DataFrame(columns = columns_yandex)
for file in files:
    df_result_yand = pd.concat([df_result_yand, pd.read_csv(file, sep=',')], axis = 0, join = 'outer')
df_result_yand.to_csv('Yandex_result_'+ str(datetime.date.today().isoformat()) + '.csv', encoding = 'utf-8')    

In [ ]:
s3 = boto3.client(service_name = 's3',
                endpoint_url = 'https://storage.yandexcloud.net',    
                aws_access_key_id = aws_access_key_id,
                aws_secret_access_key = aws_secret_access_key,
                config=Config())
with open(directory, "rb") as file:
    s3.upload_fileobj(file, 'rvaitables', 'Yandex_result_'+ str(datetime.date.today().isoformat()) + '.csv')